<a href="https://colab.research.google.com/github/mion158/Language-and-Chatbot/blob/main/Cat_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative chatbot from twitter texts

preprocessing txt files from twitter

In [10]:
from itertools import zip_longest
import re
#I have 3 txts but I'm gonna go with cat.txt for now 
files = ['cat.txt', 'love.txt', 'weather.txt']

# Defining lines as a list of each line
with open('cat.txt', 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

lines = [re.sub(r"(?:\@|https?\://)\S+", "", line).strip() for line in lines]
print(lines)

['im changing my cats name to tofu', 'imagine naming ur cat gooby instead of tofu to begin w..', "i wish she'd learn to use cutlery, she's making a right mess!", 'omg... just like my cat!', 'can you find the cat in each photo?😂', 'what cat?', 'glad we agree', "i don't like the way that cat is looking at me", 'it shen , he meows and purrs lot !', 'good cat', 'i guess someone had a busy day', 'cats are very busy creatures, always on important cat business', 'no, his name is . . . mr. tiggles 😅', 'that mr mayhem commercial where he acts like a cat knocking that stuff off the counter makes me laugh.', '─ were always there when it came to cats. brushing her knuckles across its cheek.', 'isn’t he lovely? [magnus smiled softly at the sight of the cat getting familiar, purring', 'casual photo of me and max getting ready to judge your replies to this tweet..', 'i love your cat ryan!', 'the cats in cats as presidential nominees: a thread (1/?)', 'biden is this cat:', 'she nods a bit before paddi

In [11]:
# group lines by response pair
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)
pairs = list(grouper(lines, 2))

print(pairs)

[('im changing my cats name to tofu', 'imagine naming ur cat gooby instead of tofu to begin w..'), ("i wish she'd learn to use cutlery, she's making a right mess!", 'omg... just like my cat!'), ('can you find the cat in each photo?😂', 'what cat?'), ('glad we agree', "i don't like the way that cat is looking at me"), ('it shen , he meows and purrs lot !', 'good cat'), ('i guess someone had a busy day', 'cats are very busy creatures, always on important cat business'), ('no, his name is . . . mr. tiggles 😅', 'that mr mayhem commercial where he acts like a cat knocking that stuff off the counter makes me laugh.'), ('─ were always there when it came to cats. brushing her knuckles across its cheek.', 'isn’t he lovely? [magnus smiled softly at the sight of the cat getting familiar, purring'), ('casual photo of me and max getting ready to judge your replies to this tweet..', 'i love your cat ryan!'), ('the cats in cats as presidential nominees: a thread (1/?)', 'biden is this cat:'), ('she no

In [16]:
import numpy as np
import pickle

# Building empty lists to hold sentences
input_docs = []
target_docs = []
# Building empty vocabulary sets
input_tokens = set()
target_tokens = set()

for line in pairs[:-1]:
  # Input and target sentences are separated by tabs
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below 
  # and append it to target_docs:
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words
  # and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)

input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))

In [17]:
# Create num_encoder_tokens and num_decoder_tokens:
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [18]:
for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):

  for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
    # Assign 1. for the current line, timestep, & word in encoder_input_data:
    encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    # add in conditional for handling unknown tokens (when token is not in input features dict)

  for timestep, token in enumerate(target_doc.split()):

    decoder_input_data[line, timestep, target_features_dict[token]] = 1.
    if timestep > 0:

      decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

Now training

In [19]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense, Masking
from keras.models import Model
import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Choose dimensionality
dimensionality = 256

# Choose the batch size and number of epochs:
batch_size = 10
epochs = 600

In [20]:
# Encoder training setup
encoder_inputs = Input(shape=(None, num_encoder_tokens))

encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

# Decoder training setup:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [21]:
# Building the training model:
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model:
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')

# print("Training the model:\n")
# Train the model:
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)

training_model.save('training_model.h5')

Epoch 1/600
2/2 [==============================] - 5s 1s/step - loss: 2.2741 - accuracy: 0.0125 - val_loss: 2.9936 - val_accuracy: 0.0500
Epoch 2/600
2/2 [==============================] - 0s 186ms/step - loss: 2.2290 - accuracy: 0.0578 - val_loss: 3.0154 - val_accuracy: 0.0625
Epoch 3/600
2/2 [==============================] - 0s 193ms/step - loss: 2.1120 - accuracy: 0.0250 - val_loss: 2.9685 - val_accuracy: 0.0562
Epoch 4/600
2/2 [==============================] - 0s 196ms/step - loss: 2.0167 - accuracy: 0.0391 - val_loss: 3.0061 - val_accuracy: 0.0562
Epoch 5/600
2/2 [==============================] - 0s 215ms/step - loss: 1.9840 - accuracy: 0.0391 - val_loss: 3.0343 - val_accuracy: 0.0562
Epoch 6/600
2/2 [==============================] - 0s 198ms/step - loss: 1.9696 - accuracy: 0.0484 - val_loss: 3.0466 - val_accuracy: 0.0500
Epoch 7/600
2/2 [==============================] - 0s 204ms/step - loss: 1.9545 - accuracy: 0.0406 - val_loss: 3.0582 - val_accuracy: 0.0625
Epoch 8/600
2/2 

In [23]:
from keras.models import load_model

training_model = load_model('training_model.h5')
#because working with a saved model
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]

encoder_model = Model(encoder_inputs, encoder_states)

In [24]:
latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [25]:
def decode_sequence(test_input):
  # Encode the input as state vectors.
  states_value = encoder_model.predict(test_input)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  # Populate the first token of target sequence with the start token.
  target_seq[0, 0, target_features_dict['<START>']] = 1.

  # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1).
  decoded_sentence = ''

  stop_condition = False
  while not stop_condition:
    # Run the decoder model to get possible output tokens (with probabilities) & states
    output_tokens, hidden_state, cell_state = decoder_model.predict(
      [target_seq] + states_value)

    # Choose token with highest probability
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_token = reverse_target_features_dict[sampled_token_index]
    decoded_sentence += " " + sampled_token

    # Exit condition: either hit max length or find stop token.
    if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    # Update states
    states_value = [hidden_state, cell_state]

  return decoded_sentence

Now build a generative chatbot

In [42]:
class ChatBot:
  
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")

  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
  
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on twitter cat dialogue. Let't talk about cats!\n")
    
    if user_response in self.negative_responses:
      print("Ok nevermind :(")
      return
    
    self.chat(user_response)
  
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply))
    
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix

  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    states_value = encoder_model.predict(input_matrix)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    chatbot_response = '\n'

    stop_condition = False
    while not stop_condition:
      output_tokens, hidden_state, cell_state = decoder_model.predict(
        [target_seq] + states_value)
      
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_token = reverse_target_features_dict[sampled_token_index]
      
      chatbot_response += " " + sampled_token
      
      if (sampled_token == '<END>' or len(chatbot_response) > max_decoder_seq_length):
        stop_condition = True
        
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, sampled_token_index] = 1.
      
      states_value = [hidden_state, cell_state]
      
    chatbot_response = chatbot_response.replace("<START>", "").replace("<END>", "")
      
    return chatbot_response + "\n"
  
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, it was nice talking about cats! UwU")
        return True
      
    return False

In [37]:
chatterbox = ChatBot()
chatterbox.start_chat()

Hi, I'm a chatbot trained on twitter cat dialogue.Let't talk about cats!
no
Ok nevermind :(


In [43]:
chatterbox = ChatBot()
chatterbox.start_chat()

Hi, I'm a chatbot trained on twitter cat dialogue. Let't talk about cats!
my cat's name is Meo

 imagine naming ur cat gooby instead
but she purrs and meows

 what cat ? 
my cat

 peaches ( our 20 year old geriatric
bye
Ok, it was nice talking about cats! UwU


# Mục mới